In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

In [7]:
# Separar as variáveis
X = vinhos.drop('classe', axis=1)  # Todas as colunas exceto 'classe'
y = vinhos['classe']  # Coluna 'classe' como variável-alvo


In [11]:
# Função para realizar o experimento
def realizar_experimento(random_state, k):
    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Criar o modelo KNN
    knn = KNeighborsClassifier(n_neighbors=k)

    # Treinar o modelo
    knn.fit(X_train, y_train)

    # Fazer previsões
    y_pred = knn.predict(X_test)

    # Calcular a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Experimentar com diferentes random_state e k
resultados = {}
for random_state in [42, 17, 24]:
    for k in [3, 5]:
        resultado = realizar_experimento(random_state, k)
        resultados[f'Random State {random_state} - k={k}'] = resultado

# Mostrar os resultados
for chave, valor in resultados.items():
    print(f'{chave}: {valor:.4f}')


Random State 42 - k=3: 0.7407
Random State 42 - k=5: 0.7407
Random State 17 - k=3: 0.6481
Random State 17 - k=5: 0.6667
Random State 24 - k=3: 0.7407
Random State 24 - k=5: 0.6667


In [18]:
# Função para realizar o experimento com KFold
def realizar_cross_validation(random_state, k):
    # Configurar o KFold com 10 splits
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    accuracies = []  # Para armazenar as acurácias de cada fold

    # Para cada fold
    for train_index, test_index in kf.split(X):
        # Dividir os dados em treino e teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Criar o modelo KNN com k
        knn = KNeighborsClassifier(n_neighbors=k)

        # Treinar o modelo
        knn.fit(X_train, y_train)

        # Fazer previsões
        y_pred = knn.predict(X_test)

        # Calcular a acurácia e adicionar à lista
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    # Calcular a média e o desvio padrão das acurácias
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    return mean_accuracy, std_accuracy

# Resultados para diferentes random_state e k
resultados_cv = {}
for random_state in [42, 17, 24]:
    for k in [3, 5]:
        mean_accuracy, std_accuracy = realizar_cross_validation(random_state, k)
        resultados_cv[f'Random State {random_state} - k={k}'] = (mean_accuracy, std_accuracy)

# Mostrar os resultados
for chave, (mean_accuracy, std_accuracy) in resultados_cv.items():
    print(f'{chave} - Média: {mean_accuracy:.4f}, Desvio Padrão: {std_accuracy:.4f}')


Random State 42 - k=3 - Média: 0.7036, Desvio Padrão: 0.1392
Random State 42 - k=5 - Média: 0.6641, Desvio Padrão: 0.1001
Random State 17 - k=3 - Média: 0.6987, Desvio Padrão: 0.1040
Random State 17 - k=5 - Média: 0.6876, Desvio Padrão: 0.1121
Random State 24 - k=3 - Média: 0.7127, Desvio Padrão: 0.0968
Random State 24 - k=5 - Média: 0.7127, Desvio Padrão: 0.0940


In [19]:
# Análise dos resultados
resultados_analise = {}

# Para cada random_state, comparar os resultados de k=3 e k=5
for random_state in [42, 17, 24]:
    k3_mean, k3_std = resultados_cv[f'Random State {random_state} - k=3']
    k5_mean, k5_std = resultados_cv[f'Random State {random_state} - k=5']

    # Comparar as médias de acurácia
    if k3_mean > k5_mean:
        melhor_k = 3
        melhor_media = k3_mean
        desvio_padrao = k3_std
    elif k5_mean > k3_mean:
        melhor_k = 5
        melhor_media = k5_mean
        desvio_padrao = k5_std
    else:
        melhor_k = 'Empate'
        melhor_media = k3_mean  # ou k5_mean, já que são iguais
        desvio_padrao = (k3_std + k5_std) / 2  # Média dos desvios padrão em caso de empate

    resultados_analise[f'Random State {random_state}'] = {
        'Média k=3': k3_mean,
        'Desvio Padrão k=3': k3_std,
        'Média k=5': k5_mean,
        'Desvio Padrão k=5': k5_std,
        'Melhor k': melhor_k,
        'Melhor Média': melhor_media,
        'Desvio Padrão (Melhor k)': desvio_padrao
    }

# Mostrar os resultados da análise
for random_state, resultados in resultados_analise.items():
    print(f"\nResultados para {random_state}:")
    print(f"Média k=3: {resultados['Média k=3']:.4f}, Desvio Padrão k=3: {resultados['Desvio Padrão k=3']:.4f}")
    print(f"Média k=5: {resultados['Média k=5']:.4f}, Desvio Padrão k=5: {resultados['Desvio Padrão k=5']:.4f}")
    print(f"Melhor k: {resultados['Melhor k']}")
    print(f"Melhor Média: {resultados['Melhor Média']:.4f}, Desvio Padrão (Melhor k): {resultados['Desvio Padrão (Melhor k)']:.4f}")



Resultados para Random State 42:
Média k=3: 0.7036, Desvio Padrão k=3: 0.1392
Média k=5: 0.6641, Desvio Padrão k=5: 0.1001
Melhor k: 3
Melhor Média: 0.7036, Desvio Padrão (Melhor k): 0.1392

Resultados para Random State 17:
Média k=3: 0.6987, Desvio Padrão k=3: 0.1040
Média k=5: 0.6876, Desvio Padrão k=5: 0.1121
Melhor k: 3
Melhor Média: 0.6987, Desvio Padrão (Melhor k): 0.1040

Resultados para Random State 24:
Média k=3: 0.7127, Desvio Padrão k=3: 0.0968
Média k=5: 0.7127, Desvio Padrão k=5: 0.0940
Melhor k: Empate
Melhor Média: 0.7127, Desvio Padrão (Melhor k): 0.0954
